In [1]:
import numpy as np
import cv2
from PIL import Image
from typing import Tuple, Optional, List, cast
from logging import getLogger

logger = getLogger(__name__)

INPUT_WIDTH: int = 640
INPUT_HEIGHT: int = 640

def _reorder_cv_image(src_image: np.ndarray) -> np.ndarray:
    assert src_image.ndim == 3
    assert src_image.shape[2] == 3 or src_image.shape[2] == 4
    dst_image: np.ndarray = src_image.copy()
    if dst_image.shape[2] == 3:
        dst_image = cv2.cvtColor(dst_image, cv2.COLOR_BGR2RGB)
    else:
        dst_image = cv2.cvtColor(dst_image, cv2.COLOR_BGRA2RGB)
    return dst_image

def _resize_input_image(
    image: np.ndarray, pad_value: Tuple[int, int, int] = (114, 114, 114)) -> np.ndarray:

    h, w, c = image.shape
    ratio_width: float = float(INPUT_WIDTH) / float(w)
    ratio_height: float = float(INPUT_HEIGHT) / float(h)

    ratio: float = 1.0
    dst_width: int = INPUT_WIDTH
    dst_height: int = INPUT_HEIGHT
    pad_width: int = 0
    pad_height: int = 0
    if ratio_width < ratio_height:
        # Letter box
        ratio = ratio_width
        dst_width = INPUT_WIDTH
        dst_height = min(round(h * ratio), INPUT_HEIGHT)
        pad_height = INPUT_HEIGHT - dst_height
    else:
        # Pillar box
        ratio = ratio_height
        dst_width = min(round(w * ratio), INPUT_WIDTH)
        dst_height = INPUT_HEIGHT
        pad_width = INPUT_WIDTH - dst_width
    pad_top: int = pad_height // 2
    pad_bottom: int = pad_height - pad_top
    pad_left: int = pad_width // 2
    pad_right: int = pad_width - pad_left

    logger.debug('{} {} {} {} {} {}'.format(
        dst_width, dst_height, pad_top, pad_bottom, pad_left, pad_right
    ))
    
    # Resize
    rescaled_image: np.ndarray = cv2.resize(
        src=image, dsize=(dst_width, dst_height), interpolation=cv2.INTER_LINEAR)

    # Pad
    padded_image: np.ndarray = cv2.copyMakeBorder(src=rescaled_image, 
        top=pad_top, bottom=pad_bottom, left=pad_left, right=pad_right, 
        borderType=cv2.BORDER_CONSTANT, value=pad_value)

    # Debug
    # cv2.imwrite('padded_image.png', padded_image)

    return padded_image

def _preprocess_images(images: List[np.ndarray]) -> np.ndarray:
    cv_images: List[np.ndarray] = [_resize_input_image(img) for img in images]
    np_images: List[np.ndarray] = [
        np.ascontiguousarray(img.transpose(2, 0, 1), dtype='uint8') for img in cv_images
    ]
    return np.array(np_images)

def preprocess_cv_images(images: List[np.ndarray]) -> np.ndarray:
    cv_images: List[np.ndarray] = [_reorder_cv_image(img) for img in images]
    return _preprocess_images(cv_images).astype(np.float32)

In [37]:
import numpy as np
from attrdict import AttrDict

import tritonclient.http as httpclient
from tritonclient.utils import InferenceServerException
import tritonclient.grpc.model_config_pb2 as mc

class TritonClientError(Exception):
    pass


class TritonClient():

    def __init__(self, url='localhost:8000'):
        self.request = None
        self.response = None

        try:
            self.client = httpclient.InferenceServerClient(
                url=url, verbose=False, concurrency=1
            )
        except Exception as e:
            print('could not create client: {}'.format(e))
            raise TritonClientError(str(e))

    def infer(self, image, class_count=0):
        # if self.max_batch_size > 0:
        #     image = image[np.newaxis, :]

        inputs = [httpclient.InferInput("image", (1,3,640,640), "FP32"), httpclient.InferInput("scale_factor", (1,2), "FP32")]
        inputs[0].set_data_from_numpy(image)
        inputs[1].set_data_from_numpy(np.array([[1.0, 1.0]]).astype(np.float32))

        outputs = [httpclient.InferRequestedOutput("multiclass_nms3_0.tmp_0", class_count=class_count)]

        try:
            self.request = self.client.async_infer(
                "ppyoloe_detection", inputs, request_id='0',
                model_version="1", outputs=outputs
            )
        except InferenceServerException as e:
            print('Inference failed: {}'.format(e))
            raise TritonClientError(str(e))

    def get_results(self):
        if self.request is None:
            return None

        self.response = self.request.get_result()
        output_array = self.response.as_numpy("multiclass_nms3_0.tmp_0")

        return output_array

class TritonClient_2():

    def __init__(self, url='localhost:8000'):
        self.request = None
        self.response = None

        try:
            self.client = httpclient.InferenceServerClient(
                url=url, verbose=False, concurrency=1
            )
        except Exception as e:
            print('could not create client: {}'.format(e))
            raise TritonClientError(str(e))

    def infer(self, image, class_count=0, car_count=-1):
        # if self.max_batch_size > 0:
        #     image = image[np.newaxis, :]

        inputs = [httpclient.InferInput("x", (car_count,3,192,256), "FP32")]
        inputs[0].set_data_from_numpy(image)

        outputs = [httpclient.InferRequestedOutput("sigmoid_2.tmp_0", class_count=class_count)]

        try:
            self.request = self.client.async_infer(
                "pp_attribute", inputs, request_id='0',
                model_version="1", outputs=outputs
            )
        except InferenceServerException as e:
            print('Inference failed: {}'.format(e))
            raise TritonClientError(str(e))

    def get_results(self):
        if self.request is None:
            return None

        self.response = self.request.get_result()
        output_array = self.response.as_numpy("sigmoid_2.tmp_0")

        return output_array

In [33]:
import subprocess
import time
import contextlib

from tritonclient.http import InferenceServerClient

# Define image https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver
# tag = 'nvcr.io/nvidia/tritonserver:21.10-py3'  # 6.4 GB

# Pull the image
# subprocess.call(f'docker pull {tag}', shell=True)

# Run the Triton server and capture the container ID
container_id = subprocess.check_output(
    f'docker run -d --runtime=nvidia --gpus all --rm -v /home/user/ml/tmp/triton_repo:/models -p 8000:8000 nvcr.io/nvidia/tritonserver:22.12-py3 tritonserver --model-repository=/models',
    shell=True).decode('utf-8').strip()

# Wait for the Triton server to start
triton_client = InferenceServerClient(url='localhost:8000', verbose=False, ssl=False)

# Wait until model is ready
for _ in range(10):
    with contextlib.suppress(Exception):
        assert triton_client.is_model_ready(model_name)
        break
    time.sleep(1)

In [118]:
im = cv2.imread("4.png")
# blob = cv2.dnn.blobFromImage(im, 1 / 255.0, (640, 640), swapRB=True, crop=False)
im = preprocess_cv_images([im])
tr = TritonClient()
tr.infer(im)
results_list = tr.get_results()


In [ ]:
print(results_list.shape)
print(results_list)

In [122]:
im = cv2.imread("4.png")
im_copy = cv2.imread("4.png")

height, width, _ = im.shape
height_border = width - height if width >= height else 0
width_border = height - width if height >= width else 0
print(height, width)

color_blue = (255,0,0)
color_yellow = (0,255,255)
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.7
thickness_bbox = 1
thickness_font = 2

img_list = []
points_list = []

for res in results_list:
    if res[1] > 0.5:
        start_point = (int(res[2] / 640.0 * (width + width_border) - width_border // 2), int(res[3] / 640.0 * (height + height_border) - height_border // 2)) 
        end_point = (int(res[4] / 640.0 * (width + width_border) - width_border // 2), int(res[5] / 640.0 * (height + height_border) - height_border // 2))
        # print(start_point, end_point)
        im = cv2.rectangle(im, start_point, end_point, color_blue, thickness_bbox)
        img_list.append(im_copy[max(start_point[1], 0):min(end_point[1], im.shape[0]), max(start_point[0],0):min(end_point[0],im.shape[1])])
        points_list.append((max(start_point[0],0), max(start_point[1], 0)))

INPUT_WIDTH = 256
INPUT_HEIGHT = 192

# for i, img in enumerate(img_list):
#     cv2.imwrite(str(i)+"_test.png", img)

# two type of solutions
# img_list = np.divide(preprocess_cv_images(img_list), 255.0)
img_list = np.array([(cv2.dnn.blobFromImage(i, 1.0/255.0, (256, 192), swapRB=True, crop=False))[0] for i in img_list])

tr_2 = TritonClient_2()
tr_2.infer(img_list, car_count=img_list.shape[0])
results_list_2 = tr_2.get_results()

index_to_name = ["yellow","orange","green","gray","red","blue","white","golden","brown","black"]
for i, res in enumerate(results_list_2):
    elems = heapq.nlargest(2, (res[:10]).tolist())
    indexes = [(res[:10]).tolist().index(elem) for elem in elems]
    names = [index_to_name[index] for index in indexes]
    im = cv2.putText(im, " ".join(names), points_list[i], font, font_scale, color_yellow, thickness_font)

cv2.imwrite("test.png", im)

INPUT_WIDTH = 640
INPUT_HEIGHT = 640


805 1205


In [120]:
import heapq
print(results_list_2.shape)
index_to_name = ["yellow","orange","green","gray","red","blue","white","golden","brown","black"]
for res in results_list_2:
    elems = heapq.nlargest(2, (res[:10]).tolist())
    indexes = [(res[:10]).tolist().index(elem) for elem in elems]
    print([index_to_name[index] for index in indexes])
    print([res[i] if res[i] > 0.1 else 0 for i in range(10)])

(29, 19)
['white', 'yellow']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['golden', 'yellow']
[0.14636141, 0, 0, 0, 0, 0, 0, 0.28150034, 0, 0]
['yellow', 'golden']
[0.68341696, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['yellow', 'golden']
[0.54350626, 0, 0, 0, 0, 0, 0, 0.47537476, 0, 0]
['yellow', 'gray']
[0.31309724, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['yellow', 'orange']
[0.7011129, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['yellow', 'orange']
[0.81452334, 0.12394434, 0, 0, 0, 0, 0, 0, 0.114370644, 0]
['yellow', 'black']
[0.86920476, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['yellow', 'golden']
[0.79197735, 0, 0, 0, 0, 0, 0, 0.18329674, 0, 0]
['yellow', 'golden']
[0.8210252, 0, 0, 0, 0, 0, 0, 0.20357937, 0, 0.11740911]
['yellow', 'golden']
[0.77102125, 0, 0, 0, 0, 0, 0, 0.10154492, 0, 0]
['yellow', 'golden']
[0.5720386, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['yellow', 'black']
[0.6041221, 0, 0, 0, 0, 0.13840717, 0, 0, 0.15746295, 0.2595371]
['yellow', 'golden']
[0.8444535, 0, 0, 0, 0, 0, 0, 0.33770996, 0, 0]
['yellow', 'brown']
[0.64375836, 0, 0, 0.1789826

In [31]:
subprocess.call(f'docker kill {container_id}', shell=True)

206de1c24cf3054803f8cbc89f1b0de94b60237971c338e9bef7d24287739151


0